In [1]:
# from tensorflow.keras.models import load_model
from numpy import genfromtxt, float as np_float
import logzero
import numpy as np
from Glocalx.glocalx_main.glocalx import GLocalX, shut_up_tensorflow
from Glocalx.glocalx_main.models import Rule
from Glocalx.glocalx_main.evaluators import MemEvaluator
# from Glocalx.glocalx_main.evaluators import DummyEvaluator
# from Glocalx.glocalx_main.evaluators impor/t *


import pandas as pd
import torch.nn as nn
import torch
from metadata.meta import CheckPoint
from abstract import pytorch_prediction
import torch.nn.functional as F
import json
# import pickle

C:\Users\012709558\AppData\Local\Temp\ipykernel_9528\3913041839.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import genfromtxt, float as np_float


In [2]:
data0= pd.read_csv('Path of the GMM generated data on C0 side', header=None)
data1 = pd.read_csv('Path of the GMM generated data on C1 side', header=None)
data2= pd.read_csv('Path of the GMM generated data on C2 side', header=None)
data3 = pd.read_csv('Path of the GMM generated data on C3 side', header=None)

In [4]:
dataset= pd.concat([data0, data1, data2, data3])

In [6]:
data_value = dataset.values

In [7]:
names = ['temp_hand',
 'acceleration_16_x_hand',
 'acceleration_16_y_hand',
 'acceleration_16_z_hand',
 'acceleration_6_x_hand',
 'acceleration_6_y_hand',
 'acceleration_6_z_hand',
 'gyroscope_x_hand',
 'gyroscope_y_hand',
 'gyroscope_z_hand',
 'magnetometer_x_hand',
 'magnetometer_y_hand',
 'magnetometer_z_hand',
 'temp_chest',
 'acceleration_16_x_chest',
 'acceleration_16_y_chest',
 'acceleration_16_z_chest',
 'acceleration_6_x_chest',
 'acceleration_6_y_chest',
 'acceleration_6_z_chest',
 'gyroscope_x_chest',
 'gyroscope_y_chest',
 'gyroscope_z_chest',
 'magnetometer_x_chest',
 'magnetometer_y_chest',
 'magnetometer_z_chest',
 'temp_ankle',
 'acceleration_16_x_ankle',
 'acceleration_16_y_ankle',
 'acceleration_16_z_ankle',
 'acceleration_6_x_ankle',
 'acceleration_6_y_ankle',
 'acceleration_6_z_ankle',
 'gyroscope_x_ankle',
 'gyroscope_y_ankle',
 'gyroscope_z_ankle',
 'magnetometer_x_ankle',
 'magnetometer_y_ankle',
 'magnetometer_z_ankle']
class_name  = ['motion_n', 'motion_y']

In [9]:



class SimpleNet(nn.Module):
    def __init__(self, hidden_1, dropout, output, input):
        super(SimpleNet, self).__init__()

        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, output)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x


class Net(nn.Module):

    def __init__(self, hidden_1, hidden_2, dropout, output,
                 input
                 ):

        super(Net, self).__init__()
        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_2, output)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = self.out(x)
        return x


class Layer3Net(nn.Module):

    def __init__(self, hidden_1, hidden_2, hidden_3,
                 dropout, output, input
                 ):

        super(Layer3Net, self).__init__()
        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_3, output)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        x = self.out(x)
        return x


def create_simple_net(hidden_1, dropout, output, input):
    net = SimpleNet(hidden_1, dropout, output, input)
    return net


def create_net(hidden_1=300, hidden_2=100, dropout=0.1, output=2,
               input=85):
    net = Net(hidden_1, hidden_2, dropout, output,
              input)
    return net

def create_layer3net(hidden_layer_1, hidden_layer_2, hidden_layer_3,
                     dropout_rate, n_classes, n_features):
    net = Layer3Net(hidden_layer_1, hidden_layer_2, hidden_layer_3,
                    dropout_rate, n_classes, n_features)
    return net


In [10]:
black_box = create_net(input = 39, hidden_1  = 256, hidden_2 = 256, dropout = 0.1, output = 2  )


In [11]:
ckpt_Ser = torch.load(open('path of the server model', 'rb'))

In [12]:
value_Ser = ckpt_Ser.model

In [13]:
black_box.load_state_dict(value_Ser)
black_box.eval()

Net(
  (fc1): Linear(in_features=39, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (drop): Dropout(p=0.1, inplace=False)
  (out): Linear(in_features=256, out_features=2, bias=True)
)

In [14]:
bb = pytorch_prediction(black_box)

In [15]:
z = bb.predict(data_value)

In [16]:
c_0 = 0
c_1 = 0
for i in z:
    if i==0:
        c_0+=1
    else:
        c_1+=1


In [19]:
# data = np.append(data_value, z, axis=1)
data =np.column_stack((data_value, z))

In [21]:
# local_explanations = Rule.from_json('D:/HOLDA-main/Glocalx/glocalx_main/data/dummy/dummy_rules.json', names=features_names)
data

array([[0.39332783, 0.75780409, 0.45216197, ..., 0.46888097, 0.48014984,
        0.        ],
       [0.8751758 , 0.61168397, 0.45567865, ..., 0.40401564, 0.49591677,
        1.        ],
       [0.72176824, 0.70491835, 0.43364709, ..., 0.45583451, 0.46335781,
        0.        ],
       ...,
       [0.80338885, 0.6852511 , 0.45254918, ..., 0.66768282, 0.49021561,
        0.        ],
       [0.85557475, 0.61151914, 0.38128   , ..., 0.47234891, 0.52924248,
        1.        ],
       [0.93528776, 0.68695296, 0.42635188, ..., 0.53197529, 0.43939034,
        0.        ]])

In [22]:
local_explanations = Rule.from_json('path of the rules', names=names)


In [24]:
# Set log profile: INFO for normal logging, DEBUG for verbosity
logzero.loglevel(logzero.logging.INFO)
shut_up_tensorflow()

# Load black box: optional! Use black_box = None to use the dataset labels
# black_box = load_model('D:/HOLDA-main/examples/experiments/not_hier/ServerSCkpt/global_model.h5')
# Load data and header
# data = genfromtxt('D:/HOLDA-main/GMM_data/dummy_dataset.csv', delimiter=',', names=True)
features_names = names
# tr_set = data.view(np_float).reshape(data.shape + (-1,))
tr_set = data#.reshape(data.shape + (-1,))
# Ltr_setoad local explanations
# local_explanations = Rule.from_json('data/dummy/dummy_rules.json', names=features_names)

# Create a GLocalX instance for `black_box`
glocalx = GLocalX(oracle=bb)
# Fit the model, use batch_size=128 for larger datasets
glocalx = glocalx.fit(local_explanations, tr_set, batch_size=2, name='black_box_explanations')



In [25]:
y = bb.predict(data[:, :-1]).squeeze().round()
fedality = MemEvaluator(oracle=bb)
default = int(y.mean().round())


In [26]:
alpha = 10
global_explanations_10 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_10

In [27]:
print('----------------------------------------------')
print('Alpha = 10')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1
print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 10
The fedality of glocalx with alpha = 10: 0.7691250000000001
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 2427
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 1079
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.289156626506024
----------------------------------------------


In [28]:
alpha = 20
global_explanations_20 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_20

In [29]:
print('----------------------------------------------')
print('Alpha = 20')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1

print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 20
The fedality of glocalx with alpha = 10: 0.7691250000000001
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 2396
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 1048
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.334923664122137
----------------------------------------------


In [30]:
alpha = 30
global_explanations_30 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_30

In [31]:
print('----------------------------------------------')
print('Alpha = 30')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1

print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 30
The fedality of glocalx with alpha = 10: 0.7691250000000001
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 2360
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 1012
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.359683794466402
----------------------------------------------


In [32]:
alpha = 40
global_explanations_40 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_40

In [33]:
print('----------------------------------------------')
print('Alpha = 40')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1

print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 40
The fedality of glocalx with alpha = 10: 0.7691250000000001
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 2322
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 974
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.397330595482547
----------------------------------------------


In [34]:
alpha = 50
global_explanations_50 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_50

In [35]:
print('----------------------------------------------')
print('Alpha = 50')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1

print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 50
The fedality of glocalx with alpha = 10: 0.7691250000000001
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 2267
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 919
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.44613710554951
----------------------------------------------


In [36]:
alpha = 60
global_explanations_60 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_60

In [37]:
print('----------------------------------------------')
print('Alpha = 60')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1

print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 60
The fedality of glocalx with alpha = 10: 0.7691250000000001
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 2205
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 857
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.512252042007
----------------------------------------------


In [38]:
alpha = 70
global_explanations_70 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_70

In [39]:
print('----------------------------------------------')
print('Alpha = 70')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1

print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 70
The fedality of glocalx with alpha = 10: 0.7691250000000001
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 2053
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 705
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.574468085106384
----------------------------------------------


In [40]:
alpha = 80
global_explanations_80 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_80

In [41]:
print('----------------------------------------------')
print('Alpha = 80')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1

print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 80
The fedality of glocalx with alpha = 10: 0.775325
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 1452
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 104
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.076923076923077
----------------------------------------------


In [42]:
alpha = 90
global_explanations_90 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_90

In [43]:
print('----------------------------------------------')
print('Alpha = 90')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1

print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 90
The fedality of glocalx with alpha = 10: 0.787425
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 1387
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 39
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.153846153846153
----------------------------------------------


In [44]:
alpha = 100
global_explanations_100 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_100

In [45]:
print('----------------------------------------------')
print('Alpha = 100')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1

print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 100
The fedality of glocalx with alpha = 10: 0.765575
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 1349
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 1
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.0
----------------------------------------------


In [46]:
alpha = 0
global_explanations_100 = glocalx.rules(alpha, tr_set, is_percentile=True)
test = global_explanations_100

In [47]:
print('----------------------------------------------')
print('Alpha = 0')

fedality_kjbfsk = fedality.binary_fidelity_model(units = test, x = data[:, :-1],  y = None, k=1 ,default =default)
print('The fedality of glocalx with alpha = 10:' , fedality_kjbfsk)
coverage_fk=fedality.coverage(test, data[:, :-1], targets=None, ids=None)
print('The coverage of glocalx with alpha = 10:' , sum(coverage_fk.sum(axis=0)>0)/len(data[:, :-1]))
total_rule_0 = 0
counte_0 = 0
counte_1 = 0
total_rule_1 = 1
for rule in test:

    if rule.consequence == 0:
        total_rule_0 +=  len(rule)
        counte_0 += 1
    else:
        total_rule_1 += len(rule)
        counte_1 +=1

print('the total number of rules are', counte_0+counte_1)
print('the number of rules with class = 0 are', counte_0)
print('the number of rules with class = 1 are', counte_1)
print('the avereage of the rule lenth with class = 0 are', total_rule_0/counte_0)
print('the avereage of the rule lenth with class = 1 are', total_rule_1/counte_1)
print('----------------------------------------------')

----------------------------------------------
Alpha = 0
The fedality of glocalx with alpha = 10: 0.7691250000000001
The coverage of glocalx with alpha = 10: 1.0
the total number of rules are 2459
the number of rules with class = 0 are 1348
the number of rules with class = 1 are 1111
the avereage of the rule lenth with class = 0 are 8.257418397626113
the avereage of the rule lenth with class = 1 are 8.24032403240324
----------------------------------------------
